In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from sklearn.model_selection import train_test_split
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
plotly.offline.init_notebook_mode(connected=True)

Using TensorFlow backend.


### Load Training Data

Source (Kaggle) [https://www.kaggle.com/mczielinski/bitcoin-historical-data]

In [2]:
# Load BitCoin training data
bitcoin_data = pd.read_csv('input/bitstampUSD_1-min_data_2012-01-01_to_2018-11-11.csv')

In [3]:
print(bitcoin_data.shape)
bitcoin_data.head()

(3603136, 8)


,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Preparing Data

In [4]:
bitcoin_data['date'] = pd.to_datetime(bitcoin_data['Timestamp'],unit='s').dt.date
grouped_data = bitcoin_data.groupby('date')
real_price = grouped_data['Weighted_Price'].mean()

In [5]:
data = [go.Scatter(x=real_price.index, y=real_price.values)]

plotly.offline.iplot(data)

## Building Model

In [6]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps
		if end_ix > len(sequence)-1:
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [7]:
# choose a number of time steps
n_steps = 5

# split into samples
X, y = split_sequence(real_price.values, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# split into train - test set
X_train = X[0:-365]
y_train = y[0:-365]
X_test = X[-365:]
y_test = y[-365:]

# store the test dates
test_dates = real_price.index[-365:]

In [8]:
def build_bidirectional_model(X, y):
    # define model
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=1)
    return model

### Training Data

In [9]:
bidirectional_model = build_bidirectional_model(X, y)

Epoch 1/200
2499/2499 [==============================] - 1s 579us/step - loss: 1774500.9181
Epoch 2/200
2499/2499 [==============================] - 1s 238us/step - loss: 103620.3998
Epoch 3/200
2499/2499 [==============================] - 1s 201us/step - loss: 84389.7663
Epoch 4/200
2499/2499 [==============================] - 1s 213us/step - loss: 100053.2908
Epoch 5/200
2499/2499 [==============================] - 1s 218us/step - loss: 99092.7612
Epoch 6/200
2499/2499 [==============================] - 0s 192us/step - loss: 142495.8075
Epoch 7/200
2499/2499 [==============================] - 1s 240us/step - loss: 93487.6680
Epoch 8/200
2499/2499 [==============================] - 0s 198us/step - loss: 93155.0453
Epoch 9/200
2499/2499 [==============================] - 1s 234us/step - loss: 87996.2643
Epoch 10/200
2499/2499 [==============================] - 1s 206us/step - loss: 85292.1497
Epoch 11/200
2499/2499 [==============================] - 0s 193us/step - loss: 84974.2527
Epo

2499/2499 [==============================] - 1s 207us/step - loss: 43614.0656
Epoch 92/200
2499/2499 [==============================] - 0s 198us/step - loss: 43056.7838
Epoch 93/200
2499/2499 [==============================] - 1s 200us/step - loss: 45204.7258
Epoch 94/200
2499/2499 [==============================] - 1s 240us/step - loss: 44224.5670
Epoch 95/200
2499/2499 [==============================] - 0s 198us/step - loss: 55847.6978
Epoch 96/200
2499/2499 [==============================] - 1s 217us/step - loss: 40240.4802
Epoch 97/200
2499/2499 [==============================] - 0s 191us/step - loss: 44093.4177
Epoch 98/200
2499/2499 [==============================] - 1s 201us/step - loss: 40224.6238
Epoch 99/200
2499/2499 [==============================] - 0s 195us/step - loss: 39624.1094
Epoch 100/200
2499/2499 [==============================] - 1s 228us/step - loss: 38233.2388
Epoch 101/200
2499/2499 [==============================] - 1s 210us/step - loss: 41135.6183
Epoch 102/

2499/2499 [==============================] - 0s 164us/step - loss: 33313.5054
Epoch 181/200
2499/2499 [==============================] - 0s 181us/step - loss: 33391.4465
Epoch 182/200
2499/2499 [==============================] - 0s 181us/step - loss: 34431.7607
Epoch 183/200
2499/2499 [==============================] - 0s 197us/step - loss: 35444.3064
Epoch 184/200
2499/2499 [==============================] - 0s 176us/step - loss: 33869.5024
Epoch 185/200
2499/2499 [==============================] - 1s 253us/step - loss: 34726.3842
Epoch 186/200
2499/2499 [==============================] - 1s 212us/step - loss: 41997.0629
Epoch 187/200
2499/2499 [==============================] - 1s 341us/step - loss: 56647.2545
Epoch 188/200
2499/2499 [==============================] - 0s 182us/step - loss: 32208.2552
Epoch 189/200
2499/2499 [==============================] - 0s 186us/step - loss: 33962.1452
Epoch 190/200
2499/2499 [==============================] - 0s 166us/step - loss: 41146.2871
Ep

### Prediction

In [12]:
yhat = bidirectional_model.predict(X_test, verbose=0)

In [13]:
trace_high = go.Scatter(
                x=test_dates, 
                y=y_test,
                name = "Original",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

trace_low = go.Scatter(
                x=test_dates,
                y=pd.DataFrame(yhat)[0].values,
                name = "Predicted",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

data = [trace_high,trace_low]

fig = dict(data=data)
plotly.offline.iplot(fig)